In [1]:
import pandas as pd
import numpy as np

In [2]:
beer_data = pd.read_csv("resources/clean_breweries.csv")
select_tax_data = pd.read_csv("resources/Tax_DF_Select_Columns_For_Merge.csv")

In [3]:
beer_data.head()

,Unnamed: 0,country,postalCode,province,city,address,lat,long,categories,name
0,0,US,78745,TX,Austin,407 Radam Ln,NaN,NaN,brewery,(512) Brewing Co
1,5,US,91362,CA,Westlake Village,31111 Via Colinas,NaN,NaN,brewery,101 Cider House
2,6,US,94954,CA,Petaluma,1304 Scott St,NaN,NaN,brewery,101 North Brewing Company
3,9,US,99206,WA,Spokane Valley,11616 E Montgomery Dr,NaN,NaN,brewery,12 String Brewery
4,11,US,5478,VT,Stalbans,41 Lower Newton St,44.810713,-73.083558,brewery,14th Star Brewing Co Llc


In [4]:
select_tax_data.head()

,Unnamed: 0,STATE,ZIPCODE,NUMBER OF RETURNS,NUMBER OF RETURNS WITH TOTAL INCOME,TOTAL INCOME AMOUNT,NUMBER OF RETURNS WITH SALARIES AND WAGES,SALARIES AND WAGES AMOUNT
0,1,AL,35004,5210,5210,290260,4550,234833
1,2,AL,35005,3100,3100,121690,2660,98565
2,3,AL,35006,1200,1200,57418,1040,43707
3,4,AL,35007,11940,11940,697882,10180,545418
4,5,AL,35010,7910,7910,384319,6700,266658


In [5]:
beer_data_renamed = beer_data.rename(columns={
    "postalCode": "ZIPCODE",
    "province": "STATE"
})
beer_data_renamed.head()

,Unnamed: 0,country,ZIPCODE,STATE,city,address,lat,long,categories,name
0,0,US,78745,TX,Austin,407 Radam Ln,NaN,NaN,brewery,(512) Brewing Co
1,5,US,91362,CA,Westlake Village,31111 Via Colinas,NaN,NaN,brewery,101 Cider House
2,6,US,94954,CA,Petaluma,1304 Scott St,NaN,NaN,brewery,101 North Brewing Company
3,9,US,99206,WA,Spokane Valley,11616 E Montgomery Dr,NaN,NaN,brewery,12 String Brewery
4,11,US,5478,VT,Stalbans,41 Lower Newton St,44.810713,-73.083558,brewery,14th Star Brewing Co Llc


In [6]:
# The tax data has significantly more entries than the beer data.
print(len(beer_data_renamed.ZIPCODE))
print(len(select_tax_data.ZIPCODE))

5714
29872


In [10]:
beer_data_renamed.dtypes

Unnamed: 0      int64
country        object
ZIPCODE        object
STATE          object
city           object
address        object
lat           float64
long          float64
categories     object
name           object
dtype: object

In [13]:
beer_data_renamed[beer_data_renamed.ZIPCODE.str.contains(",")]

,Unnamed: 0,country,ZIPCODE,STATE,city,address,lat,long,categories,name
29,76,US,"97227, 97212",OR,Portland,3901 N Williams Ave,45.550914,-122.667015,brewery,5th Quadrant
673,2920,US,"49301, 49357",MI,Ada,418 Ada Dr,42.955520,-85.483830,brewery,Gravel Bottom Craft Brewery Supply


In [14]:
beer_data_renamed.ZIPCODE = beer_data_renamed.ZIPCODE.str.split(",").str[0]

In [15]:
beer_data_renamed[beer_data_renamed.ZIPCODE.str.contains(",")]

,Unnamed: 0,country,ZIPCODE,STATE,city,address,lat,long,categories,name


In [11]:
select_tax_data.dtypes

Unnamed: 0                                    int64
STATE                                        object
ZIPCODE                                       int64
NUMBER OF RETURNS                             int64
NUMBER OF RETURNS WITH TOTAL INCOME           int64
TOTAL INCOME AMOUNT                           int64
NUMBER OF RETURNS WITH SALARIES AND WAGES     int64
SALARIES AND WAGES AMOUNT                     int64
dtype: object

In [16]:
beer_data_renamed["ZIPCODE"] = beer_data_renamed["ZIPCODE"].astype("int")
beer_data_renamed.dtypes

Unnamed: 0      int64
country        object
ZIPCODE         int32
STATE          object
city           object
address        object
lat           float64
long          float64
categories     object
name           object
dtype: object

In [19]:
# For the merge, we'll want to keep all columns of tax data, with multiple entries for zip codes that contain more than one brewery.
# So we need an outer join

merged_df = pd.merge(select_tax_data, beer_data_renamed, on= ["ZIPCODE", "STATE"], how="outer")
merged_df.head(200)

# Columns to drop: "Unnamed: 0_x", "Unnamed: 0_y", "country"

,Unnamed: 0_x,STATE,ZIPCODE,NUMBER OF RETURNS,NUMBER OF RETURNS WITH TOTAL INCOME,TOTAL INCOME AMOUNT,NUMBER OF RETURNS WITH SALARIES AND WAGES,SALARIES AND WAGES AMOUNT,Unnamed: 0_y,country,city,address,lat,long,categories,name
0,1.0,AL,35004,5210.0,5210.0,290260.0,4550.0,234833.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,AL,35005,3100.0,3100.0,121690.0,2660.0,98565.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,AL,35006,1200.0,1200.0,57418.0,1040.0,43707.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,AL,35007,11940.0,11940.0,697882.0,10180.0,545418.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,AL,35010,7910.0,7910.0,384319.0,6700.0,266658.0,15973.0,US,Alexander City,"Via Giuseppe Garibaldi, 54",NaN,NaN,brewery,Azienda Agricola Francesco De Tacchi
5,6.0,AL,35014,1620.0,1620.0,75228.0,1410.0,57364.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7.0,AL,35016,7130.0,7130.0,366988.0,5680.0,262108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8.0,AL,35019,890.0,890.0,38993.0,740.0,30460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9.0,AL,35020,9050.0,9050.0,251540.0,7930.0,207009.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10.0,AL,35022,9800.0,9800.0,551747.0,8270.0,432275.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
cleaner_merged_df = merged_df.drop(labels = ["Unnamed: 0_x", "Unnamed: 0_y", "country"], axis=1)
cleaner_merged_df.head()

,STATE,ZIPCODE,NUMBER OF RETURNS,NUMBER OF RETURNS WITH TOTAL INCOME,TOTAL INCOME AMOUNT,NUMBER OF RETURNS WITH SALARIES AND WAGES,SALARIES AND WAGES AMOUNT,city,address,lat,long,categories,name
0,AL,35004,5210.0,5210.0,290260.0,4550.0,234833.0,NaN,NaN,NaN,NaN,NaN,NaN
1,AL,35005,3100.0,3100.0,121690.0,2660.0,98565.0,NaN,NaN,NaN,NaN,NaN,NaN
2,AL,35006,1200.0,1200.0,57418.0,1040.0,43707.0,NaN,NaN,NaN,NaN,NaN,NaN
3,AL,35007,11940.0,11940.0,697882.0,10180.0,545418.0,NaN,NaN,NaN,NaN,NaN,NaN
4,AL,35010,7910.0,7910.0,384319.0,6700.0,266658.0,Alexander City,"Via Giuseppe Garibaldi, 54",NaN,NaN,brewery,Azienda Agricola Francesco De Tacchi


In [21]:
cleaner_merged_df.to_csv("resources/merged_data.csv")